<a href="https://colab.research.google.com/github/Bala534/WIDS-Datathon-2021/blob/main/WIDS_Datathon_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import keras
import tensorboard
import tensorflow as tf
import random as rd

In [250]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score,confusion_matrix,classification_report,roc_auc_score,roc_curve,auc

In [251]:
data = pd.read_csv('/content/TrainingWiDS2021.csv')
data.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,...,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,...,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [252]:
data.shape

(9439, 181)

In [253]:
data.isna().any() # Check for NaN under an entire DataFrame:

Unnamed: 0                     False
encounter_id                   False
hospital_id                    False
age                             True
bmi                             True
                               ...  
immunosuppression               True
leukemia                        True
lymphoma                        True
solid_tumor_with_metastasis     True
diabetes_mellitus               True
Length: 181, dtype: bool

In [254]:
data.isna().sum()

Unnamed: 0                        0
encounter_id                      0
hospital_id                       0
age                             381
bmi                            1714
                               ... 
immunosuppression                 1
leukemia                          1
lymphoma                          1
solid_tumor_with_metastasis       1
diabetes_mellitus                 1
Length: 181, dtype: int64

In [255]:
data.isnull().sum().sum() # Count the NaN under an entire DataFrame:

588645

In [256]:
data.columns

Index(['Unnamed: 0', 'encounter_id', 'hospital_id', 'age', 'bmi',
       'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source',
       ...
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'aids', 'cirrhosis',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'],
      dtype='object', length=181)

In [257]:
data.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,9439.000000,9439.000000,9439.000000,9058.000000,7725.000000,9439.000000,9303.000000,9439.000000,9439.000000,9439.0,7786.000000,3322.000000,9269.000000,9395.000000,9439.000000,9439.000000,3217.000000,7121.000000,7253.000000,2758.000000,9373.000000,9373.000000,9376.000000,9373.000000,8477.000000,9439.000000,7655.000000,9439.000000,9429.000000,2758.000000,2758.000000,2758.000000,2758.000000,9404.000000,7164.000000,9356.00000,1087.000000,9439.000000,7481.000000,3339.000000,...,1939.000000,1939.000000,1940.000000,1940.000000,3273.000000,3273.000000,509.000000,509.000000,1771.000000,1771.000000,1609.000000,1609.000000,1364.000000,1364.000000,1765.000000,1765.000000,3344.000000,3344.000000,3348.000000,3348.000000,3338.000000,3338.000000,3047.000000,3047.000000,1750.000000,1750.000000,1767.000000,1767.000000,1747.000000,1747.000000,1458.000000,1458.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000
mean,4720.000000,212468.712893,102.456934,62.472842,29.827418,0.236042,169.870080,103.801780,0.639364,0.0,86.196686,2.797953,187.477398,610.246416,0.253205,0.026698,1.128816,25.539952,1.551428,0.563054,3.633949,5.614638,0.000320,4.390483,155.925469,98.229579,33.169484,0.167920,94.912504,43.260986,43.260986,123.126831,7.347823,31.526053,138.083752,36.42084,1565.263168,0.302680,11.589332,80.932016,...,11.076741,11.047860,33.236340,33.115979,1.558340,1.440605,3.123969,3.111821,186.940147,186.570299,4.154319,4.134307,137.605572,137.491935,12.806065,12.764868,46.650598,39.205593,7.386166,7.317984,164.164900,100.472817,295.049280,222.777543,46.151509,44.281354,7.334514,7.320857,153.597956,127.486448,242.531535,232.138740,0.000424,0.017800,0.013562,0.047468,0.007841,0.005616,0.026912,0.236703
std,2724.948929,38034.680807,30.576800,16.507772,8.397404,0.424671,10.721735,16.447873,2.131302,0.0,25.600630,0.658332,84.636890,493.712560,0.434870,0.161207,2.232036,20.660617,1.485407,0.248652,0.839758,1.133063,0.017886,1.263630,84.223594,31.268728,6.721519,0.373815,40.370748,12.005139,12.005139,71.794378,0.089854,15.528441,5.084512,0.82818,1344.659034,0.459442,6.723268,22.602841,...,2.287738,2.295524,6.525713,6.588485,0.972934,0.789128,3.117094,3.101945,91.004823,91.086263,0.784581,0.769225,5.868815,5.883618,6.587150,6.545033,14.351535,10.576916,0.076608,0.105024,100.223659,54.954640,128.151609,114.624686,13.406992,12.794979,0.094819,0.097001,101.181760,79.163288,124.991761,120.437770,0.020584,0.132232,0.115671,0.212648,0.088204,0.074731,0.161836,0.425081
min,1.000000,147009.000000,4.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-0.224306,0.0,38.600000,1.200000,101.000000,0.040000,0.000000,0.000000,0.100000,4.000000,0.300000,0.210000,1.000000,1.00000

In [258]:
data.describe(include="all")

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,9439.000000,9439.000000,9439.000000,9058.000000,7725.000000,9439.000000,9236,9436,9303.000000,8556,9421,9439.000000,9439,9439,9439.000000,9439.0,7786.000000,3322.000000,9269.000000,9395.000000,9439.000000,9439.000000,3217.000000,7121.000000,7253.000000,2758.000000,9373.000000,9373.000000,9376.000000,9373.000000,8477.000000,9439.000000,7655.000000,9439.000000,9429.000000,2758.000000,2758.000000,2758.000000,2758.000000,9404.000000,...,1939.000000,1939.000000,1940.000000,1940.000000,3273.000000,3273.000000,509.000000,509.000000,1771.000000,1771.000000,1609.000000,1609.000000,1364.000000,1364.000000,1765.000000,1765.000000,3344.000000,3344.000000,3348.000000,3348.000000,3338.000000,3338.000000,3047.000000,3047.000000,1750.000000,1750.000000,1767.000000,1767.000000,1747.000000,1747.000000,1458.000000,1458.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000,9438.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,6,2,NaN,9,5,NaN,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,Caucasian,M,NaN,Emergency Department,Accident & Emergency,NaN,admit,Med-Surg ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,7836,5131,NaN,4133,5440,NaN,9329,5436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4720.000000,212468.712893,102.456934,62.472842,29.827418,0.236042,NaN,NaN,169.870080,NaN,NaN,103.801780,NaN,NaN,0.639364,0.0,86.196686,2.797953,187.477398,610.246416,0.253205,0.026698,1.128816,25.539952,1.551428,0.563054,3.633949,5.614638,0.000320,4.390483,155.925469,98.229579,33.169484,0.167920,94.912504,43.260986,43.260986,123.126831,7.347823,31.526053,...,11.076741,11.047860,33.236340,33.115979,1.558340,1.440605,3.123969,3.111821,186.940147,186.570299,4.154319,4.134307,137.605572,137.491935,12.806065,12.764868,46.650598,39.205593,7.386166,7.317984,164.164900,100.472817,295.049280,222.777543,46.151509,44.281354,7.334514,7.320857,153.597956,127.486448,242.531535,232.138740,0.000424,0.017800,0.013562,0.047468,0.007841,0.005616,0.026912,0.236703
std,2724.948929,38034.680807,30.576800,16.5

In [259]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9439 entries, 0 to 9438
Columns: 181 entries, Unnamed: 0 to diabetes_mellitus
dtypes: float64(162), int64(13), object(6)
memory usage: 13.0+ MB


In [260]:
del data['ethnicity']

In [261]:
del data['hospital_admit_source']

In [262]:
del data['icu_admit_source']

In [263]:
del data['icu_stay_type']

In [264]:
del data['icu_type']

In [265]:
data.columns = data.columns.str.strip()

In [266]:
data['gender'] = data.gender.map(dict(M=1, F=0))

In [267]:
data = data.fillna(method="ffill")

In [268]:
data.isna().sum().sum()

8162

In [269]:
data = data.fillna(method="bfill")

In [270]:
data.isna().sum().sum()

0

In [271]:
# Implementing the model

In [272]:
from sklearn.ensemble import RandomForestRegressor

In [273]:
x = data.loc[ : , data.columns != 'diabetes_mellitus']
y = data['diabetes_mellitus']

In [274]:
model = RandomForestRegressor()
model.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [279]:
df = pd.read_csv('/content/UnlabeledWiDS2021.csv')

In [280]:
df.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,1,144740,10141,72,NaN,0,Caucasian,F,152.4,Floor,Accident & Emergency,82,admit,Med-Surg ICU,0.015278,0,NaN,2.8,110.0,104.01,0,0,1.9,44.0,1.49,NaN,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,NaN,NaN,NaN,NaN,31.0,...,26.0,14.5,14.5,42.8,42.8,NaN,NaN,NaN,NaN,173.0,173.0,4.9,4.9,132.0,132.0,5.6,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,2,141990,10141,86,NaN,0,Caucasian,F,175.3,Emergency Department,Accident & Emergency,82,admit,Med-Surg ICU,0.000000,0,NaN,NaN,117.0,106.01,0,0,NaN,19.0,0.92,NaN,4.0,6.0,0.0,5.0,73.0,116.0,NaN,0,41.0,NaN,NaN,NaN,NaN,53.0,...,NaN,NaN,NaN,NaN,NaN,2.2,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
2,3,142038,10141,72,NaN,0,Caucasian,F,162.6,Floor,Floor,82,admit,Med-Surg ICU,0.003472,0,NaN,3.2,302.0,109.01,0,0,0.9,24.0,0.77,NaN,4.0,6.0,0.0,5.0,84.0,104.0,34.8,0,117.0,NaN,NaN,NaN,NaN,38.0,...,32.0,11.8,11.8,36.6,36.6,2.9,2.4,NaN,NaN,243.0,243.0,3.6,3.6,141.0,141.0,6.6,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
3,4,138628,10141,66,NaN,0,Caucasian,M,177.8,Floor,Floor,82,admit,Med-Surg ICU,0.884028,0,NaN,NaN,113.0,501.05,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,110.0,NaN,0,61.0,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
4,5,141682,10141,89,NaN,0,Caucasian,M,170.2,Direct Admit,Accident & Emergency,82,admit,Med-Surg ICU,0.013194,0,NaN,3.4,117.0,106.01,0,0,0.4,26.0,1.50,NaN,4.0,6.0,0.0,5.0,99.0,34.0,33.0,0,136.0,NaN,NaN,NaN,NaN,35.0,...,30.0,10.7,10.7,33.0,33.0,1.1,1.1,NaN,NaN,154.0,154.0,4.0,4.0,133.0,133.0,6.2,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0


In [296]:
df.shape

(10234, 175)

In [282]:
del df['ethnicity']

In [283]:
del df['hospital_admit_source']

In [284]:
del df['icu_admit_source']

In [285]:
del df['icu_stay_type']

In [286]:
del df['icu_type']

In [287]:
df.columns = df.columns.str.strip()

In [288]:
df['gender'] = df.gender.map(dict(M=1, F=0))

In [289]:
df = df.fillna(method="ffill")

In [290]:
df.isna().sum().sum()

1588

In [291]:
df = df.fillna(method="bfill")

In [293]:
df.isna().sum().sum()

0

In [297]:
df.drop(df.tail(795).index, 
        inplace = True) 

In [298]:
y_pred = model.predict(df)
y_pred

array([0.36, 0.2 , 0.16, ..., 0.11, 0.22, 0.41])

In [299]:
roc_auc_score(y,y_pred)

0.4956038064186253

In [301]:
output = pd.DataFrame({'encounter_id':df['encounter_id'],'diabetes_milletus':y_pred})
output

,encounter_id,diabetes_milletus
0,144740,0.36
1,141990,0.20
2,142038,0.16
3,138628,0.30
4,141682,0.46
...,...,...
9434,135424,0.31
9435,143540,0.23
9436,138804,0.11
9437,143701,0.22
